In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd

In [2]:
save_dir = '/home/jl277509/Documents/translation_only'

In [3]:
# instead of printing, add in table
from pathlib import Path
import pandas as pd

sub_dirs = {'SOr': ['/neurospin/dico/jlaval/Output/option1/SOr_left_UKB40/', ['troiani_embeddings/Left_OFC/full_values.json'], 'full_weighted_auc'],
            'FIP': ['/neurospin/dico/jlaval/Output/option1/FIP_right_UKB40/', ['FIP_right_embeddings/Right_FIP/full_values.json'], 'full_auc'],
            'SC-sylv': ['/neurospin/dico/jlaval/Output/option1/SC-sylv_left_UKB40/', [f'hcp_isomap_embeddings/Isomap_central_left_dim{k}/full_values.json' for k in range(1,7)], 'full_r2']}

sub_dirs = {'SOr': ['/neurospin/dico/jlaval/Output/3_combinations_ablation/SOr_left_UKB40', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'test_weighted_auc'],
            'FIP': ['/neurospin/dico/jlaval/Output/3_combinations_ablation/FIP_right_UKB40', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'test_auc'],
            'SC-sylv': ['/neurospin/dico/jlaval/Output/3_combinations_ablation/SC-sylv_left_UKB40', [f'hcp_isomap_random_embeddings/Isomap_central_left_dim{k}/full_values.json' for k in range(1,7)], 'full_r2']}

# trim ablation
sub_dirs = {'SOr': ['/neurospin/dico/jlaval/Output/2_single_augm_ablation/SOr_left_UKB40', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'test_weighted_auc'],
            'FIP': ['/neurospin/dico/jlaval/Output/2_single_augm_ablation/FIP_right_UKB40', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'test_auc'],
            'SC-sylv': ['/neurospin/dico/jlaval/Output/2_single_augm_ablation/SC-sylv_left_UKB40', [f'hcp_isomap_random_embeddings/Isomap_central_left_dim{k}/full_values.json' for k in range(1,7)], 'full_r2']}
root_names = ['1_trimdepth', '2_trimdepth_constant', '3_trimextremities', '4_trimextremities_constant']
# cutin cutout ablation

sub_dirs = {'SOr': ['/neurospin/dico/jlaval/Output/1_cutin_cutout_param_ablation/SOr_left_UKB40', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'test_weighted_auc'],
            'FIP': ['/neurospin/dico/jlaval/Output/1_cutin_cutout_param_ablation/FIP_right_UKB40', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'test_auc'],
            'SC-sylv': ['/neurospin/dico/jlaval/Output/1_cutin_cutout_param_ablation/SC-sylv_left_UKB40', [f'hcp_isomap_random_embeddings/Isomap_central_left_dim{k}/full_values.json' for k in range(1,7)], 'full_r2']}
root_names = ['10_cutout_keep_all_p30', '11_cutout_keep_all_p50', '12_cutout_keep_all_p70', '13_cutout_keep_bottom_p30', '14_cutout_keep_bottom_p50', '15_cutout_keep_bottom_p70',
              '16_cutout_keep_bottom_p100', '1_cutin_no_keep', '2_cutin_keep_all_p30', '3_cutin_keep_all_p50', '5_cutin_keep_bottom_p30',
              '6_cutin_keep_bottom_p50', '7_cutin_keep_bottom_p70', '8_cutin_keep_bottom_p100', '9_cutout_no_keep', '4_cutin_keep_all_p70'] # '4_cutin_keep_all_p70' : RERUN

# translation only
sub_dirs = {'SOr': ['/neurospin/dico/jlaval/Output/4_translation_only/SOr_left_UKB40', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'test_weighted_auc'],
            'FIP': ['/neurospin/dico/jlaval/Output/4_translation_only/FIP_right_UKB40', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'test_auc'],
            'SC-sylv': ['/neurospin/dico/jlaval/Output/4_translation_only/SC-sylv_left_UKB40', [f'hcp_isomap_random_embeddings/Isomap_central_left_dim{k}/full_values.json' for k in range(1,7)], 'full_r2']}
root_names = ['1_translation_only']

#sub_dir = '/neurospin/dico/jlaval/Output/option1/SOr_left_UKB40/'
#sub_dir = '/neurospin/dico/jlaval/Output/option1/FIP_right_UKB40/'
#sub_dir = '/neurospin/dico/jlaval/Output/option1/SC-sylv_left_UKB40/'
#auc_file = 'hcp_isomap_random_embeddings/Isomap_central_left_dim5/full_values.json'
#metric = 'full_r2'
#root_names = ['1_cutin_no-keep-bottom', '2_cutin_cutout_no-keep-bottom', '3_cutin_cutout_keep-bottom', '4_cutin_cutout_trimdepth_no-keep-bottom', '5_cutin_cutout_trimdepth_keep-bottom', '6_all_augmentations_no-keep-bottom', '7_all_augmentations_keep-bottom']
#root_names=['1_all_augmentations_keep_bottom']

# store everything globally
l_model_glob = []
l_perf_glob = []
l_region_glob = []
l_task_glob = []


for region, (sub_dir, auc_files, metric) in sub_dirs.items(): 
    for auc_file in auc_files:
        models = os.listdir(sub_dir)
        models = [elem for elem in models if 'config' not in elem and 'multirun' not in elem]
        models.sort()
        task = Path(auc_file).parts[-2]
        for name in root_names:
            #l_models = [elem for elem in models if elem.startswith(name)]
            l_models = [elem for elem in models if name in elem]
            l_perfs = []
            for model in l_models:
                try:
                    with open(os.path.join(sub_dir, model, auc_file), 'r') as f:
                        perf = json.load(f)
                        if isinstance(perf[metric], float):
                            value = perf[metric]
                        else:
                            value = perf[metric][0]
                        l_perfs.append(value)
                except Exception:
                    print(f'{os.path.join(sub_dir, model, auc_file)} not found') ## NB : can't continue if not all files found ...
            l_model_glob.append(l_models)
            l_perf_glob.append(l_perfs)
            l_region_glob.append([region for _ in range(len(l_perfs))])
            l_task_glob.append([task for _ in range(len(l_perfs))])

In [4]:
# reformat lists and put inside DataFrame
def flatten_sum(matrix):
    return sum(matrix, [])
cols = ['model', 'region', 'task', 'score']
values = np.array([flatten_sum(l_model_glob), flatten_sum(l_region_glob), flatten_sum(l_task_glob), flatten_sum(l_perf_glob)]).T
df = pd.DataFrame(data=values, columns=cols)


In [5]:
df

,model,region,task,score
0,1_translation_onlyname10-52-06_163,SOr,Left_OFC,0.6387228471195256
1,1_translation_onlyname11-15-03_208,SOr,Left_OFC,0.6583317244609154
2,1_translation_onlyname11-18-52_221,SOr,Left_OFC,0.662842757941373
3,1_translation_onlyname12-11-52_177,SOr,Left_OFC,0.6390530220091621
4,1_translation_onlyname13-32-40_155,SOr,Left_OFC,0.652341950744045
5,1_translation_onlyname10-52-06_140,FIP,Right_FIP,0.681664737654321
6,1_translation_onlyname10-52-06_249,FIP,Right_FIP,0.683304398148148
7,1_translation_onlyname11-15-33_50,FIP,Right_FIP,0.7058256172839505
8,1_translation_onlyname12-02-13_35,FIP,Right_FIP,0.742283950617284
9,1_translation_onlyname13-10-55_155,FIP,Right_FIP,0.7400655864197531


In [6]:
df.to_csv(f'{save_dir}.csv', index=False)

In [7]:
## ONLY IF THERE ARE 5 MODELS
nb_runs = 5

# fsimplify model names and aggregate
# TODO : more flexible code !!
lmodels = df['model'].tolist()
#lmodels = [model.split('bottom')[0]+'bottom' for model in lmodels] ## won't generalise ...
lmodels = [model.split('name')[0] for model in lmodels]
df1 = df.copy()
df1['model']=lmodels

df1['group'] = df.index // nb_runs  # Creates group numbers (0,0,0,0,0, 1,1,1,1,1) # TODO : what if not 5 models ?
df1['score'] = pd.to_numeric(df1['score'], errors='coerce')
averaged_df1 = df1.groupby('group').agg({'model': 'first', 'region': 'first', 'task': 'first', 'score': ['mean', 'std']}).reset_index() ## ADD STD here
averaged_df1.columns = ['group', 'model', 'region', 'task', 'score_mean', 'score_std']
averaged_df1['score_mean']*=100
averaged_df1['score_std']*=100
averaged_df1 = averaged_df1.round(decimals=1)

In [46]:
averaged_df1.loc[averaged_df1['model']=='9_cutout_no_keep']

,group,model,region,task,score_mean,score_std
15,15,9_cutout_no_keep,SOr,Left_OFC,66.1,NaN
31,31,9_cutout_no_keep,FIP,Right_FIP,79.1,NaN


In [18]:
# SAVE SUMMARY RESULTS
averaged_df1.to_csv(f'{save_dir}_summary.csv', index=False)

In [8]:
averaged_df1

,group,model,region,task,score_mean,score_std
0,0,1_translation_only,SOr,Left_OFC,65.0,1.1
1,1,1_translation_only,FIP,Right_FIP,71.1,2.9
2,2,1_translation_only,SC-sylv,Isomap_central_left_dim1,16.1,7.5
3,3,1_translation_only,SC-sylv,Isomap_central_left_dim2,8.9,6.1
4,4,1_translation_only,SC-sylv,Isomap_central_left_dim3,20.3,12.3
5,5,1_translation_only,SC-sylv,Isomap_central_left_dim4,5.1,6.0
6,6,1_translation_only,SC-sylv,Isomap_central_left_dim5,9.8,4.2
7,7,1_translation_only,SC-sylv,Isomap_central_left_dim6,8.8,4.0
